# RAG with DIY-Search and PHI3-mini

In [1]:
import os
os.environ['HF_HOME'] = '/run/cache/' # DON'T FORGET THIS, ELSE DISK CACHE WILL RUN OUT OF SPACE WHEN YOU DOWNLOAD XL MODEL

import json
import requests
import minsearch

In [2]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs = requests.get(docs_url).json()

In [3]:
documents = []

for course_dict in docs:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
index = minsearch.Index(text_fields=["question", "text", "section"],
                        keyword_fields=["course"]
)

index.fit(documents)

In [5]:
def search(q):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=q,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

# Microsoft - phi3-mini

In [7]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         100G   61G   40G  61% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  100G   61G   40G  61% /run
tmpfs            14G     0   14G   0% /dev/shm
/dev/nvme2n1    9.8G  1.9G  7.9G  20% /home/jovyan
tmpfs            14G  120K   14G   1% /home/jovyan/.saturn
tmpfs            14G   12K   14G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           7.7G   12K  7.7G   1% /proc/driver/nvidia
tmpfs           7.7G   11M  7.7G   1% /run/nvidia-persistenced/socket
tmpfs           7.7G     0  7.7G   0% /proc/acpi
tmpfs           7.7G     0  7.7G   0% /sys/firmware


In [8]:
import torch 
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline 

torch.random.manual_seed(0) 
model = AutoModelForCausalLM.from_pretrained( 
    "microsoft/Phi-3-mini-128k-instruct",  
    device_map="cuda",  
    torch_dtype="auto",  
    trust_remote_code=True,  
) 

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct") 

config.json:   0%|          | 0.00/3.48k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
def build_prompt(question, context):
    prompt = """You are a chatbot answering frequently asked questions for an online course. 
    Provide ANSWER to the QUESTION based on the CONTEXT given below. 
    Only state the facts from the CONTEXT else respond that information is not available.
    
    QUESTION = {question}
    CONTEXT = {context}
    ANSWER
    """
    prompt_output = prompt.format(question = question, context=context)
    return prompt_output

In [14]:
def llm(prompt):
    messages = [ 
    {"role": "system", "content": prompt}
    ]
    pipe = pipeline( 
        "text-generation", 
        model=model, 
        tokenizer=tokenizer, 
    ) 

    generation_args = { 
        "max_new_tokens": 500, 
        "return_full_text": False, 
        "temperature": 0.0, 
        "do_sample": False, 
    } 

    output = pipe(messages, **generation_args) 
    result = output[0]['generated_text'].strip()

    return result

In [15]:
def rag_response(q):
    context = search(q)
    prompt = build_prompt(q, context)
    answer = llm(prompt)
    return answer

In [16]:
query = "Do i need to know python to pass the course?"
rag_response(query)

'You do not need to know Python to pass the course. The course materials and instructions are provided in a way that does not require prior knowledge of Python.'